In [12]:
import torch

# Understanding Your Dataset and Gaussian Distribution

## Dataset Overview
- **Observations**: You're working with a dataset that contains several observations of a single scalar variable (let's call it `x`). If you have `N` observations, your dataset might look something like this: `x_1, x_2, ..., x_N`.

## Assumptions and Definitions
- **Independence**: It's assumed that each observation in your dataset is drawn independently from every other one.
- **Identically Distributed**: All observations come from the same Gaussian distribution, meaning they share the same mean (μ) and variance (σ²).
- **I.I.D.**: This concept is so common that it has its own abbreviation: i.i.d., which stands for independent and identically distributed.

## Gaussian Distribution
- **Mean and Variance**: The Gaussian distribution is characterized by its mean (μ) and variance (σ²), but in this case, we don't know these values yet. They are the parameters we want to estimate from our dataset.

## Joint Probability and Likelihood Function
- **Product of Marginals**: Since the dataset is i.i.d., the joint probability of the dataset is the product of the marginal probabilities for each individual observation.
- **Likelihood Function**: This joint probability is also known as the likelihood function when viewed as a function of the parameters (μ and σ²) given the data:

$$
{L}(\mu, \sigma^2 | \mathbf{X}) = p(\mathbf{X} | \mu, \sigma^2) = \prod_{n=1}^N N(x_n | \mu, \sigma^2).
$$

Here, `N(x_n | μ, σ²)` represents the normal distribution for an individual observation, and the product over all `N` observations gives us the likelihood of the dataset given the parameters.

## Objective
- **Parameter Estimation**: Your goal is to find the values of μ and σ² that maximize the likelihood function, which would mean that the parameters are the most probable given the data observed.


# Generating a Dataset from a Normal Distribution with PyTorch

In this step, we're going to create a synthetic dataset by drawing samples from a normal (Gaussian) distribution. This is a common initial step in many statistical simulations and machine learning tasks, where you need a dataset to work with.

We'll use the PyTorch library, which is a powerful tool for both deep learning and general scientific computing. Here's how we do it:

1. **Define the Distribution Parameters**:
   - The `mean` is the central tendency of the distribution.
   - The `std` (standard deviation) measures the spread of the distribution.
   - These are the (arbitrary) parameters we will estimate

2. **Generate Samples**:
   - We use the `torch.normal` function to draw samples from a normal distribution with the defined `mean` and `std`.
   - The `size` argument in `torch.normal` specifies the number of samples to generate. Here, we ask for 100,000 samples.

By executing this code, we create a `tensor` object in PyTorch that holds our dataset. This dataset can then be used for further analysis, such as parameter estimation or machine learning model training.


In [13]:
mean = 10.2
std = 2.3

# ask for 100,000 samples
dataset = torch.normal(mean, std, size=(100_000,))
dataset

tensor([11.4836,  9.6073,  8.9638,  ..., 14.5447,  9.0194,  8.9777])

# Why Use the Log Likelihood?

In statistical modeling, especially when dealing with probability distributions, we often need to estimate parameters that best explain our data. The likelihood function, which is the probability of the data given the parameters, is central to this estimation process. However, when we work with the likelihood directly, we face the challenge of dealing with very small numbers because probabilities can be extremely small. This can lead to numerical underflow, where the computer rounds small numbers down to zero.

To avoid this, we use the log likelihood. The logarithm is a monotonic transformation, meaning it preserves the order of values: if one value is larger than another, its logarithm is also larger. Therefore, maximizing the log likelihood is equivalent to maximizing the likelihood itself. Additionally, the logarithm has convenient mathematical properties that turn products into sums, making calculations easier and more numerically stable.

# Derivation of the Log Likelihood for a Normal Distribution

The likelihood function for a set of observations assuming a normal distribution is the product of the probability density function (PDF) of each observation. Let's derive the log-likelihood from this starting point:

## Step 1: The Normal Distribution PDF
The PDF of a normally distributed random variable $x$ is given by:

$$
p(x|\mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left( -\frac{(x-\mu)^2}{2\sigma^2} \right)
$$

## Step 2: The Likelihood Function
For $N$ independent observations ${x_1, x_2, \ldots, x_N}$, the likelihood function is the product of the individual PDFs:

$$
{L}(\mu, \sigma^2 | \mathbf{x}) = \prod_{n=1}^N p(x_n|\mu, \sigma^2)
$$

## Step 3: Log Likelihood
Taking the natural logarithm of the likelihood function, we convert the product into a sum:

$$
\ln {L}(\mu, \sigma^2 | \mathbf{x}) = \sum_{n=1}^N \ln p(x_n|\mu, \sigma^2)
$$

## Step 4: Substituting the PDF
Substitute the normal distribution PDF into the log likelihood:

$$
\ln {L}(\mu, \sigma^2 | \mathbf{x}) = \sum_{n=1}^N \ln \left( \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left( -\frac{(x_n-\mu)^2}{2\sigma^2} \right) \right)
$$

## Step 5: Simplifying the Expression
Using properties of logarithms, we simplify the equation:

$$
\ln {L}(\mu, \sigma^2 | \mathbf{x}) = \sum_{n=1}^N \left( -\frac{(x_n-\mu)^2}{2\sigma^2} - \ln \sqrt{2\pi\sigma^2} \right)
$$

$$
\ln {L}(\mu, \sigma^2 | \mathbf{x}) = -\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2 - \frac{N}{2} \ln (2\pi\sigma^2)
$$

Finally, we separate the constant terms from the variable terms:

$$
\ln {L}(\mu, \sigma^2 | \mathbf{x}) = -\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2 - \frac{N}{2} \ln \sigma^2 - \frac{N}{2} \ln (2\pi)
$$

This is the log likelihood function for the normal distribution that we use for maximum likelihood estimation.


# Finding the Maximum Likelihood Estimate of the Mean

Given the log likelihood function for a normal distribution:

$$
\ln {L}(\mu, \sigma^2 | \mathbf{x}) = -\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n - \mu)^2 - \frac{N}{2} \ln \sigma^2 - \frac{N}{2} \ln(2\pi)
$$

We want to maximize this function with respect to $\mu$

## Derivative of the Log Likelihood Function

The first step is to take the derivative of the log likelihood with respect to $\mu$:

$$
\frac{d}{d\mu} \ln {L}(\mu, \sigma^2 | \mathbf{x}) = \frac{d}{d\mu} \left( -\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n - \mu)^2 \right)
$$

Since $\sigma^2$ is a constant, and the derivative of a sum is the sum of the derivatives, we can simplify this to:

$$
\frac{d}{d\mu} \ln {L}(\mu, \sigma^2 | \mathbf{x}) = -\frac{1}{2\sigma^2} \sum_{n=1}^N \frac{d}{d\mu} (x_n - \mu)^2
$$

The derivative of $(x_n - \mu)^2$ with respect to $\mu$ is $-2(x_n - \mu)$. Plugging this back into the equation gives us:

$$
\frac{d}{d\mu} \ln {L}(\mu, \sigma^2 | \mathbf{x}) = \frac{1}{\sigma^2} \sum_{n=1}^N (x_n - \mu)
$$

## Setting the Derivative to Zero

To find the maximum of the log likelihood, we set its derivative equal to zero:

$$
\frac{1}{\sigma^2} \sum_{n=1}^N (x_n - \mu) = 0
$$

Multiplying through by $\sigma^2$ and then dividing by $N$ gives us:

$$
\sum_{n=1}^N (x_n - \mu) = 0
$$

$$
\sum_{n=1}^N x_n - N\mu = 0
$$

Solving for $\mu$ we get:

$$
\mu = \frac{1}{N} \sum_{n=1}^N x_n
$$

This is the maximum likelihood estimate for the mean $ \mu$, denoted as $\mu_{ML}$, which is simply the average of all observations.



# Finding the Maximum Likelihood Estimate of the Variance

After finding the maximum likelihood estimate for the mean $\mu$, we proceed to estimate the variance $\sigma^2$. We use the previously obtained $\mu_{ML}$ in this process.

## Starting with the Log Likelihood Function

We have the log likelihood function given by:

$$
\ln {L}(\mu, \sigma^2 | \mathbf{x}) = -\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n - \mu)^2 - \frac{N}{2} \ln \sigma^2 - \frac{N}{2} \ln(2\pi)
$$

## Derivative with Respect to $\sigma^2$

To maximize the log likelihood with respect to $\sigma^2$, we take its derivative and set it to zero. The derivative of the log likelihood with respect to $\sigma^2$ is:

$$
\frac{d}{d\sigma^2} \ln {L}(\mu, \sigma^2 | \mathbf{x}) = \frac{d}{d\sigma^2} \left( -\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n - \mu)^2 - \frac{N}{2} \ln \sigma^2 \right)
$$

Since the first term is the sum of $ -\frac{1}{2\sigma^2}$ times each squared difference, its derivative is the sum of $ \frac{1}{2\sigma^4}$ times each squared difference. The derivative of the second term $ -\frac{N}{2} \ln \sigma^2 $ with respect to $ \sigma^2 $ is $ -\frac{N}{2\sigma^2} $.

Combining these gives us:

$$
\frac{d}{d\sigma^2} \ln {L}(\mu, \sigma^2 | \mathbf{x}) = \frac{1}{2\sigma^4} \sum_{n=1}^N (x_n - \mu)^2 - \frac{N}{2\sigma^2}
$$

## Setting the Derivative to Zero

Now, we set the derivative to zero and solve for $ \sigma^2$:

$$
\frac{1}{2\sigma^4} \sum_{n=1}^N (x_n - \mu)^2 - \frac{N}{2\sigma^2} = 0
$$

Multiplying through by $ 2\sigma^4$ gives us:

$$
\sum_{n=1}^N (x_n - \mu)^2 = N\sigma^2
$$

Dividing through by $N$ gives us the maximum likelihood estimate for the variance $ \sigma^2$:

$$
\sigma^2_{ML} = \frac{1}{N} \sum_{n=1}^N (x_n - \mu_{ML})^2
$$

This is the sample variance, which is the mean of the squared differences between each observation and the sample mean $ \mu_{ML}$. It's important to note that when using $ \mu_{ML} $ for the mean in the variance formula, we're applying the maximum likelihood estimate for the mean that we derived earlier.



In [14]:
estimated_mean = torch.mean(dataset)
estimated_variance = torch.mean((dataset - estimated_mean) ** 2)

estimated_mean, torch.sqrt(estimated_variance)

(tensor(10.2071), tensor(2.3099))

# Unbiased Estimation of Variance

As we can see, the estimations are quite good. Play around with the sample size to see how this changes.
While the maximum likelihood estimator for the mean of a Gaussian distribution is unbiased, the MLE for the variance is biased. This bias occurs because the MLE uses the sample mean, which itself is an estimator, and not the true population mean.

## Expectation of MLE Variance

The expectation of the maximum likelihood estimator of the variance is shown to be:

$$
\mathbb{E}[\sigma^2_{ML}] = \left( \frac{N - 1}{N} \right) \sigma^2
$$

This indicates that the MLE for variance systematically underestimates the true variance.

## Correction for Unbiased Variance

To correct for this bias, we use the following unbiased estimator:

$$
\hat{\sigma}^2 = \frac{N}{N - 1} \sigma^2_{ML} = \frac{1}{N - 1} \sum_{n=1}^N (x_n - \mu_{ML})^2
$$

This adjustment uses $ N - 1 $ in the denominator instead of $N$, which compensates for the bias introduced by the MLE.

The code snippet provided demonstrates how to calculate this unbiased variance in Python using NumPy. By using `len(dataset) - 1`, we ensure that our variance estimate is unbiased and a more accurate reflection of the true population variance.


In [15]:
unbiased_variance = torch.sum((dataset - estimated_mean) ** 2) / (dataset.size(0) - 1)

torch.sqrt(unbiased_variance), torch.sqrt(estimated_variance)

(tensor(2.3099), tensor(2.3099))